# Chapter 47: Pipeline Performance Optimization

As codebases grow and microservices multiply, CI/CD pipelines can become bottlenecks rather than accelerators. Builds taking 30+ minutes destroy the tight feedback loops essential for agile development, while inefficient resource utilization drives cloud costs exponentially. Pipeline performance optimization requires a systematic approach: identifying bottlenecks through timing analysis, implementing intelligent caching at multiple layers, parallelizing independent operations, and right-sizing compute resources to match workload characteristics without over-provisioning.

This chapter establishes strategies for sub-5-minute build pipelines, covering Docker layer caching, dependency resolution acceleration, test parallelization, and cost optimization techniques that maintain velocity while controlling infrastructure spend.

## 47.1 Pipeline Bottleneck Analysis

Before optimizing, identify where time is actually spent.

### Build Timing Analysis

**GitHub Actions Timing:**
```yaml
# .github/workflows/build.yml
jobs:
  build:
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      
      - name: Setup Java
        uses: actions/setup-java@v3
        with:
          java-version: '17'
          distribution: 'temurin'
      
      - name: Build with Timing
        run: |
          echo "::group::Dependency Resolution"
          time ./mvnw dependency:go-offline
          echo "::endgroup::"
          
          echo "::group::Compilation"
          time ./mvnw compile -DskipTests
          echo "::endgroup::"
          
          echo "::group::Testing"
          time ./mvnw test
          echo "::endgroup::"
          
          echo "::group::Packaging"
          time ./mvnw package -DskipTests
          echo "::endgroup::"
```

**Explanation:**
The `time` command prefixes each Maven phase, outputting real/user/system time. GitHub Actions collapsible groups (`::group::`/`::endgroup::`) organize logs by phase. Typical bottlenecks appear in:
- **Dependency resolution**: Downloading artifacts (network I/O)
- **Compilation**: CPU-bound, scales with code size
- **Testing**: Often the longest phase, I/O and CPU bound
- **Docker build**: Layer caching misses, large context transfers

### Profiling Tools

**Maven Build Profile:**
```bash
./mvnw clean package -DskipTests \
  -Dmaven.ext.class.path=/usr/share/maven/maven-profile-*.jar \
  --builder smart \
  -T 4
```

**Gradle Build Scan:**
```bash
./gradlew build --scan
# Generates https://scans.gradle.com/s/unique-id
# Shows task execution timeline, cache hits, dependency resolution time
```

**Explanation:**
Gradle Build Scans provide visualization of build execution, showing which tasks ran in parallel, which were cached, and where time was spent. The `--parallel` flag enables parallel project builds for multi-module projects.

## 47.2 Caching Strategies

Effective caching eliminates redundant work across builds.

### Dependency Caching

**Maven Cache (GitHub Actions):**
```yaml
- name: Cache Maven Dependencies
  uses: actions/cache@v3
  with:
    path: |
      ~/.m2/repository
      !~/.m2/repository/com/company  # Exclude SNAPSHOTs
    key: ${{ runner.os }}-maven-${{ hashFiles('**/pom.xml') }}
    restore-keys: |
      ${{ runner.os }}-maven-
```

**Explanation:**
The cache key includes the OS and a hash of all `pom.xml` files. If any POM changes, the cache misses. `restore-keys` provides partial matching—if the exact key misses, it falls back to the most recent Maven cache, then downloads only changed dependencies. The exclusion `!~/.m2/repository/com/company` ensures internal SNAPSHOTs aren't cached between builds (they change frequently).

**Npm Cache:**
```yaml
- name: Cache Node Modules
  uses: actions/cache@v3
  with:
    path: |
      ~/.npm
      node_modules
    key: ${{ runner.os }}-node-${{ hashFiles('**/package-lock.json') }}
    restore-keys: |
      ${{ runner.os }}-node-
```

**Layered Caching Strategy:**
```yaml
# Multi-layer cache for better hit rates
- name: Cache Gradle Wrapper
  uses: actions/cache@v3
  with:
    path: ~/.gradle/wrapper
    key: ${{ runner.os }}-gradle-wrapper-${{ hashFiles('**/gradle-wrapper.properties') }}

- name: Cache Gradle Dependencies
  uses: actions/cache@v3
  with:
    path: ~/.gradle/caches
    key: ${{ runner.os }}-gradle-${{ hashFiles('**/*.gradle*', '**/gradle.properties') }}
    restore-keys: |
      ${{ runner.os }}-gradle-
```

### Build Cache (Gradle Build Cache)

**Remote Build Cache:**
```groovy
// gradle/build-cache-settings.gradle
buildCache {
    local {
        enabled = true
        directory = "${rootDir}/.gradle/build-cache"
        removeUnusedEntriesAfterDays = 30
    }
    
    remote(HttpBuildCache) {
        url = 'https://cache.company.com/cache/'
        credentials {
            username = System.getenv('CACHE_USERNAME')
            password = System.getenv('CACHE_PASSWORD')
        }
        enabled = true
        push = isCiServer
        
        // Only cache if build succeeds
        allowUntrustedServer = false
        allowInsecureProtocol = false
    }
}
```

**Explanation:**
Gradle's build cache stores task outputs (compiled classes, test results, processed resources). If inputs haven't changed (determined by hashing), it restores outputs from cache rather than re-executing tasks. Remote caches share build outputs across CI runners, so if one runner compiles a module, others download the result instead of recompiling.

### Docker Layer Caching

**GitHub Actions Docker Cache:**
```yaml
- name: Set up Docker Buildx
  uses: docker/setup-buildx-action@v3

- name: Build and Push
  uses: docker/build-push-action@v5
  with:
    context: .
    push: true
    tags: company/app:latest
    cache-from: type=gha
    cache-to: type=gha,mode=max
```

**Explanation:**
`type=gha` uses GitHub Actions cache backend for Docker layers. Each Dockerfile instruction creates a layer; if the instruction and context haven't changed, Docker reuses the cached layer. `mode=max` exports all layers (including intermediate) for maximum cache efficiency.

**Registry Cache (Alternative):**
```yaml
cache-from: type=registry,ref=company/app:cache
cache-to: type=registry,ref=company/app:cache,mode=max
```

**Explanation:**
Storing cache in the container registry (`company/app:cache` tag) rather than GitHub Actions cache provides larger storage limits and persistence across workflow runs, though with slightly higher latency.

## 47.3 Parallel Execution

### Test Parallelization

**Maven Surefire Parallel Execution:**
```xml
<!-- pom.xml -->
<plugin>
    <groupId>org.apache.maven.plugins</groupId>
    <artifactId>maven-surefire-plugin</artifactId>
    <version>3.1.2</version>
    <configuration>
        <parallel>methods</parallel>
        <threadCount>4</threadCount>
        <forkCount>2</forkCount>
        <reuseForks>true</reuseForks>
        <argLine>-Xmx512m</argLine>
    </configuration>
</plugin>
```

**Explanation:**
- **parallel=methods**: Runs test methods in parallel within same JVM
- **threadCount=4**: Uses 4 threads per fork
- **forkCount=2**: Creates 2 separate JVM processes
- **Total parallelism**: 2 JVMs × 4 threads = 8 concurrent tests
- **reuseForks=true**: Reuses JVMs between test classes (faster than restarting JVM)

**JUnit 5 Parallel Configuration:**
```properties
# src/test/resources/junit-platform.properties
junit.jupiter.execution.parallel.enabled=true
junit.jupiter.execution.parallel.mode.default=concurrent
junit.jupiter.execution.parallel.mode.classes.default=concurrent
junit.jupiter.execution.parallel.config.strategy=dynamic
junit.jupiter.execution.parallel.config.dynamic.factor=2
```

**Resource Locking (Prevent Test Interference):**
```java
@Test
@ResourceLock("Database")  // Only one test using "Database" runs at a time
void testDatabaseOperation() {
    // Modifies shared database state
}

@Test
@ResourceLock(value = "Database", mode = ResourceAccessMode.READ)  
void testDatabaseRead() {
    // Can run concurrently with other READ locks
}
```

### Pipeline Job Parallelization

**GitHub Actions Matrix Strategy:**
```yaml
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      fail-fast: false
      matrix:
        service: [payment, order, inventory, user]
        test-suite: [unit, integration]
        exclude:
          - service: inventory
            test-suite: integration  # Skip slow integration tests for inventory
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Test ${{ matrix.service }} - ${{ matrix.test-suite }}
        run: |
          cd services/${{ matrix.service }}
          npm run test:${{ matrix.test-suite }}
```

**Explanation:**
The matrix creates 3 services × 2 suites = 6 parallel jobs (minus 1 exclusion). `fail-fast: false` ensures all jobs run even if one fails, useful for identifying which specific service/test combination broke.

**DAG (Directed Acyclic Graph) Dependencies:**
```yaml
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - run: ./mvnw package -DskipTests
      - uses: actions/upload-artifact@v3
        with:
          name: jar-artifact
          path: target/*.jar

  test-unit:
    needs: build  # Runs after build completes
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/download-artifact@v3
        with:
          name: jar-artifact
      - run: ./mvnw test -Dtest=UnitTest

  test-integration:
    needs: build
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/download-artifact@v3
        with:
          name: jar-artifact
      - run: ./mvnw test -Dtest=IntegrationTest

  deploy:
    needs: [test-unit, test-integration]  # Waits for both
    runs-on: ubuntu-latest
    steps:
      - run: echo "Deploying..."
```

**Explanation:**
The DAG ensures efficient parallelization: build runs first, then unit and integration tests run in parallel (both need the build artifact), and deploy waits for both test jobs. This reduces total pipeline time from sequential (build → unit → integration → deploy) to build + max(unit, integration) + deploy.

## 47.4 Docker Optimization

### Multi-Stage Builds

**Optimized Dockerfile:**
```dockerfile
# Stage 1: Build with all dependencies
FROM eclipse-temurin:17-jdk-alpine AS builder
WORKDIR /app

# Copy only dependency files first (better caching)
COPY pom.xml .
COPY src ./src

# Download dependencies (cached layer)
RUN ./mvnw dependency:go-offline -B

# Build application
RUN ./mvnw package -DskipTests -B

# Stage 2: Runtime with JRE only
FROM eclipse-temurin:17-jre-alpine AS runtime
WORKDIR /app

# Create non-root user
RUN addgroup -S appgroup && adduser -S appuser -G appgroup

# Copy only the JAR from builder
COPY --from=builder /app/target/*.jar app.jar

# Set ownership
RUN chown -R appuser:appgroup /app
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=3s \
  CMD wget -q --spider http://localhost:8080/actuator/health || exit 1

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
```

**Explanation:**
- **Stage 1 (builder)**: Uses full JDK, includes Maven, compiles code. This layer is heavy but doesn't ship to production.
- **Stage 2 (runtime)**: Uses slim JRE (~60MB vs ~200MB), no build tools, only runtime user. The final image contains only the JAR and JRE.
- **Caching**: Copying `pom.xml` before `src` means dependency download is cached unless dependencies change, not on every code change.

### Layer Ordering

**Optimize Layer Cache Hits:**
```dockerfile
# BAD - Changes on every build
COPY . /app
RUN npm install
RUN npm run build

# GOOD - Stable layers first
COPY package*.json /app/
RUN npm ci --only=production  # Cached unless dependencies change

COPY . /app
RUN npm run build  # Only runs when code changes
```

**Explanation:**
Docker caches layers based on the instruction and context hash. If `COPY . /app` comes before `npm install`, every code change invalidates the cache, forcing npm install to re-run. By copying only package files first, the expensive `npm ci` operation is cached and reused unless `package.json` changes.

### BuildKit Features

**Mount Cache (Persistent Caching Between Builds):**
```dockerfile
# syntax=docker/dockerfile:1
FROM node:18-alpine AS builder
WORKDIR /app

COPY package*.json ./
RUN --mount=type=cache,target=/root/.npm \
    npm ci

COPY . .
RUN --mount=type=cache,target=/app/.next/cache \
    npm run build
```

**Explanation:**
`--mount=type=cache` creates persistent cache directories that survive between builds. The npm cache is stored separately from layers, speeding up `npm ci`. The `.next/cache` directory (Next.js build cache) persists incremental build data.

**SSH Mount (Private Dependencies):**
```dockerfile
RUN --mount=type=ssh,id=github \
    git clone git@github.com:company/private-repo.git
```

**Build Command:**
```bash
docker build --ssh default=${SSH_AUTH_SOCK} .
```

**Explanation:**
The SSH mount allows accessing private Git repositories during build without copying SSH keys into the image. The SSH agent socket is mounted temporarily during the RUN instruction, then discarded.

## 47.5 Network Optimization

### Artifact Repository Proximity

**Regional Caching (Nexus/Artifactory):**
```xml
<!-- settings.xml -->
<mirrors>
  <mirror>
    <id>company-nexus</id>
    <name>Company Nexus - US-East</name>
    <url>https://nexus.useast.company.com/repository/maven-public/</url>
    <mirrorOf>central</mirrorOf>
  </mirror>
</mirrors>

<profiles>
  <profile>
    <id>ci</id>
    <properties>
      <!-- Use regional mirror based on runner location -->
      <nexus.url>https://nexus.${env.AWS_REGION}.company.com</nexus.url>
    </properties>
  </profile>
</profiles>
```

**Explanation:**
Place artifact repositories (Nexus, Artifactory, npm registries) in the same region as CI runners to reduce latency. A dependency download from Maven Central (US) to a runner in Europe adds 100-200ms latency per artifact; a regional mirror reduces this to <10ms.

### Git Clone Optimization

**Shallow Clones:**
```yaml
- uses: actions/checkout@v4
  with:
    fetch-depth: 0  # Full history (slow)

- uses: actions/checkout@v4
  with:
    fetch-depth: 1  # Shallow clone (fast, only latest commit)
```

**Git LFS (Large File Storage) Optimization:**
```yaml
- uses: actions/checkout@v4
  with:
    lfs: true
    fetch-depth: 1

- name: Cache LFS
  uses: actions/cache@v3
  with:
    path: .git/lfs
    key: lfs-${{ hashFiles('.lfs-assets-id') }}
```

**Explanation:**
Shallow clones (`fetch-depth: 1`) download only the latest commit, reducing clone time from minutes to seconds for large repositories. However, this breaks operations requiring history (like `git describe` or comparing with base branch). Use fetch depth 0 only when necessary.

## 47.6 Resource Right-Sizing

### CPU and Memory Tuning

**Maven Memory Settings:**
```bash
# Analyze memory usage first
./mvnw package -DskipTests \
  -Dmaven.ext.class.path=/usr/share/maven/maven-profile-*.jar \
  -Dprofile=true

# Then optimize
export MAVEN_OPTS="-Xms1g -Xmx2g -XX:+UseG1GC -XX:MaxGCPauseMillis=200"
```

**Node.js Memory:**
```bash
# Default is 512MB, increase for large builds
node --max-old-space-size=4096 ./node_modules/.bin/ng build --prod
```

**Docker Resource Constraints:**
```yaml
# .github/workflows/build.yml
jobs:
  build:
    runs-on: ubuntu-latest
    container:
      image: maven:3.9-eclipse-temurin-17
      options: --cpus 4 --memory 8g
```

**Explanation:**
Right-sizing prevents:
- **Under-allocation**: OutOfMemory errors, excessive GC pauses, slow builds due to swapping
- **Over-allocation**: Wasted costs, noisy neighbor problems in shared CI environments

The `options` flag passes Docker resource constraints to the container. For CPU-intensive builds (compilation), ensure the CPU limit matches the parallelization (e.g., 4 CPUs for `-T 4` Maven builds).

### Runner Sizing (GitHub Actions/GitLab CI)

**Self-Hosted Runner Tiers:**
```yaml
# Small jobs (linting, unit tests)
small-job:
  runs-on: [self-hosted, small]  # 2 vCPU, 4GB RAM

# Medium jobs (build)
medium-job:
  runs-on: [self-hosted, medium]  # 4 vCPU, 8GB RAM

# Large jobs (integration tests with databases)
large-job:
  runs-on: [self-hosted, large]  # 8 vCPU, 16GB RAM
```

**Explanation:**
Label runners by size and route jobs appropriately. Unit tests need minimal resources; integration tests with Docker Compose need large runners. This prevents small jobs from waiting for large runners while avoiding over-provisioning costs for simple tasks.

## 47.7 Cost Optimization

### Spot Instances for CI

**AWS EC2 Spot Runners:**
```yaml
# GitHub Actions with EC2 Spot
jobs:
  start-runner:
    runs-on: ubuntu-latest
    outputs:
      label: ${{ steps.start-ec2-runner.outputs.label }}
      ec2-instance-id: ${{ steps.start-ec2-runner.outputs.ec2-instance-id }}
    steps:
      - name: Start EC2 runner
        id: start-ec2-runner
        uses: machulav/ec2-github-runner@v2
        with:
          mode: start
          github-token: ${{ secrets.GH_PERSONAL_ACCESS_TOKEN }}
          ec2-image-id: ami-1234567890abcdef0
          ec2-instance-type: c5.4xlarge
          subnet-id: subnet-1234567
          security-group-id: sg-1234567
          spot: true  # Use spot instances (60-90% cheaper)
          spot-max-price: 0.50  # Maximum spot price willing to pay
```

**Explanation:**
Spot instances provide significant cost savings (60-90%) for CI workloads, which are fault-tolerant (if the instance terminates, the job restarts on another runner). The `spot-max-price` sets a ceiling; if spot price exceeds this, the instance terminates.

### Artifact Lifecycle Management

**S3 Lifecycle Policies:**
```yaml
# Terraform configuration for artifact bucket
resource "aws_s3_bucket_lifecycle_configuration" "artifacts" {
  bucket = aws_s3_bucket.artifacts.id

  rule {
    id     = "archive-old-builds"
    status = "Enabled"

    filter {
      prefix = "builds/"
    }

    transition {
      days          = 30
      storage_class = "STANDARD_IA"  # Infrequent Access (cheaper)
    }

    transition {
      days          = 90
      storage_class = "GLACIER"  # Archive (much cheaper)
    }

    expiration {
      days = 365  # Delete after 1 year
    }
  }
}
```

**Explanation:**
CI artifacts (JARs, Docker images, test reports) accumulate rapidly. Lifecycle policies automatically move old artifacts to cheaper storage classes:
- **Standard**: Frequent access (first 30 days)
- **Standard-IA**: 40% cheaper, retrieval fee (30-90 days)
- **Glacier**: 80% cheaper, slow retrieval (90+ days)
- **Expiration**: Automatic deletion after 365 days for compliance

### Docker Image Cleanup

**Registry Garbage Collection:**
```bash
# GitLab Container Registry
curl --request DELETE \
  --header "PRIVATE-TOKEN: $TOKEN" \
  "https://gitlab.com/api/v4/projects/12345/registry/repositories/67/tags/old-tag"

# Retention policy (keep last 10 tags per image)
docker run -it --rm \
  -e RETAIN_COUNT=10 \
  -e REGISTRY_URL=https://registry.company.com \
  -e REGISTRY_USER=$USER \
  -e REGISTRY_PASS=$PASS \
  registry-cleanup-tool
```

## 47.8 Test Execution Optimization

### Test Impact Analysis

**Running Only Affected Tests:**
```bash
# Git diff to find changed files
CHANGED_FILES=$(git diff --name-only HEAD~1)

# Run tests only for changed modules (Maven)
./mvnw test -pl $(echo $CHANGED_FILES | grep -o 'services/[^/]*' | sort -u | tr '\n' ',')

# Jest only changed files
npx jest --onlyChanged --changedSince=origin/main
```

**Explanation:**
Rather than running the entire test suite (30+ minutes), run only tests related to changed code. Maven's `-pl` (projects list) flag limits testing to specific modules. Jest's `--onlyChanged` uses Git to determine which test files to run based on imports.

### Parallel Test Execution with Database Sharding

```java
@TestExecutionListeners({
    DependencyInjectionTestExecutionListener.class,
    TransactionalTestExecutionListener.class
})
@SpringBootTest
@TestPropertySource(properties = {
    "spring.datasource.url=jdbc:postgresql://localhost:5432/test_db_${random.uuid}"
})
public class IntegrationTest {
    // Each test class gets its own database schema
}
```

**Explanation:**
Database contention limits test parallelization. By giving each test class (or thread) a unique database schema (using `${random.uuid}` or Testcontainers dynamic ports), tests run truly in parallel without lock contention. Cleanup drops the schema after tests complete.

### Testcontainers Optimization

```java
@TestConfiguration
public class TestcontainersConfig {
    
    // Reuse container across test classes (faster)
    @Container
    public static PostgreSQLContainer<?> postgres = new PostgreSQLContainer<>("postgres:15")
        .withDatabaseName("test")
        .withReuse(true);  // Enable container reuse
    
    @DynamicPropertySource
    static void properties(DynamicPropertyRegistry registry) {
        registry.add("spring.datasource.url", postgres::getJdbcUrl);
    }
}
```

**Explanation:**
Testcontainers start fresh containers for each test by default (slow). With `withReuse(true)` and Ryuk (resource reaper) disabled, containers survive between test classes if configuration matches, reducing startup time from 30s to <1s for subsequent tests.

---

## Chapter Summary and Preview

This chapter established performance optimization as a critical discipline for maintaining fast, cost-effective CI/CD pipelines. We examined bottleneck analysis techniques using timing wrappers and build scans to identify where pipeline time is actually spent, distinguishing between network-bound (dependency downloads), CPU-bound (compilation), and I/O-bound (testing) phases. Multi-layer caching strategies—including dependency caches keyed on lock files, Gradle build caches shared across CI runners, and Docker layer caches using BuildKit—eliminate redundant work by preserving outputs between builds.

Parallel execution at multiple levels—test methods within JVMs, parallel CI jobs via matrix strategies, and DAG-based job orchestration—maximizes resource utilization while respecting dependencies. Docker optimization through multi-stage builds reduces final image sizes by 70-80% by discarding build tools, while intelligent layer ordering ensures dependency installation remains cached across code changes. Network optimization via regional artifact repositories and shallow Git clones reduces latency, and resource right-sizing ensures adequate memory and CPU for build tools without over-provisioning.

Cost optimization strategies leverage spot instances for fault-tolerant CI workloads, S3 lifecycle policies for artifact archival, and test impact analysis to run only affected tests rather than full suites. Testcontainers reuse and database sharding enable parallel integration testing without contention, reducing test phase duration from hours to minutes.

**Key Takeaways:**
- Implement multi-layer caching: cache dependency downloads based on lock file hashes, use Gradle/Maven build caches for compiled outputs, and leverage Docker BuildKit cache mounts for package manager caches between builds.
- Structure Dockerfiles to maximize layer caching: copy dependency manifests (package.json, pom.xml) before source code, install dependencies, then copy and build source; this ensures dependency layers remain cached unless dependencies change.
- Use matrix builds and DAG dependencies to parallelize independent jobs, reducing wall-clock time from sequential sums to critical path duration (typically build + max(test stages) + deploy).
- Right-size CI runners: provide 4-8 vCPUs for compilation (to utilize parallel Maven/Gradle builds), 16GB+ RAM for integration tests with Testcontainers, and use small runners for linting/documentation tasks.
- Implement test impact analysis to run only tests affected by code changes in PR builds, reserving full test suites for main branch builds or nightly runs.
- Use spot instances or preemptible VMs for CI workloads to reduce compute costs by 60-90%, implementing retry logic for jobs terminated due to spot preemption.

**Next Chapter Preview:**
Chapter 48: Security in CI/CD addresses the protection of software supply chains and delivery pipelines. We will explore supply chain security including Software Bills of Materials (SBOM) generation and verification, container image signing with Cosign and Sigstore, vulnerability scanning integration into pipelines, secrets management without hardcoding, and compliance frameworks for regulated industries. The chapter covers zero-trust pipeline architectures, artifact provenance verification, and automated security policy enforcement, ensuring that velocity does not compromise security posture.